# Análise com SQL Avançado
## U.S. EPA Food Commodity Intake Database (FCID)
### [https://fcid.foodrisk.org/](https://fcid.foodrisk.org/)

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

# Importando Tabelas do FCID

In [2]:
DROP TABLE IF EXISTS Crop_Group;
DROP TABLE IF EXISTS FCID_Description;
DROP TABLE IF EXISTS Recipes;
DROP TABLE IF EXISTS Intake;

CREATE TABLE Crop_Group (
  CGN VARCHAR(2),
  CGL VARCHAR(6),
  Crop_Group_Description VARCHAR(80),
  PRIMARY KEY (CGL)
) AS SELECT
  CGN, CGL, Crop_Group_Description
FROM CSVREAD('../data/food-intake/basics/FCID_Cropgroup_Description.csv');

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code),
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../data/food-intake/basics/FCID_Code_Description.csv');

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method),
  FOREIGN KEY(FCID_Code)
    REFERENCES FCID_Description(FCID_Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

# Visualizando as Tabelas

In [3]:
SELECT * FROM Crop_Group LIMIT 10;

In [4]:
SELECT * FROM FCID_Description LIMIT 10;

In [5]:
SELECT * FROM Recipes LIMIT 10;

In [6]:
SELECT * FROM Intake LIMIT 10;

# Métricas

Considere que a tabela Intake registra alimentos consumidos por 1.489 pessoas. Considere as seguintes métricas para um alimento:

| Métrica | Descrição |
| --- | --- |
| Popularidade | número de pessoas (dentre as 1.489) que consumiram o alimento |
| Intake_Sum | total consumido do alimento pelas 1.489 pessoas em gramas |
| Intake_AVG | média de consumo do alimento em gramas |
| Intake_AVG_BW | média de consumo do alimento x peso da pessoa |
| Recipes | número de receitas (dentre as 7.154 receitas) que têm o produto como ingrediente |

## 1)  Construa uma View que apresente essas métricas por produto

* Veja exemplo em: `/data/food-intake/computed/commodity-profile.csv`
* Importante: esta tabela foi feita com um número maior de registros, portanto os valores não serão iguais aos seus

In [ ]:
CREATE VIEW Food_Metrics AS
SELECT 
  p.FCID_Code,
  p.Popularidade,
  i_s.Intake_Sum,
  i_a.Intake_AVG,
  i_abw.Intake_AVG_BW,
  r.Recipes
FROM 
  (SELECT FCID_Code, COUNT(DISTINCT SeqN) AS Popularidade FROM Intake GROUP BY FCID_Code) p
JOIN
  (SELECT FCID_Code, SUM(Intake) AS Intake_Sum FROM Intake GROUP BY FCID_Code) i_s ON p.FCID_Code = i_s.FCID_Code
JOIN
  (SELECT FCID_Code, AVG(Intake) AS Intake_AVG FROM Intake GROUP BY FCID_Code) i_a ON p.FCID_Code = i_a.FCID_Code
JOIN
  (SELECT FCID_Code, AVG(Intake_BW) AS Intake_AVG_BW FROM Intake GROUP BY FCID_Code) i_ab ON  p.FCID_Code = i_abw.FCID_Code
JOIN
  (SELECT FCID_Code, COUNT(*) AS Recipes FROM Recipes GROUP BY FCID_Code) r ON  p.FCID_Code = r.FCID_Code;
  
SELECT 
  i.FCID_Code,
  COUNT(DISTINCT SeqN) AS Popularidade,
  SUM(i.Intake) as Intake_Sum,
  AVG(i.Intake) AS Intake_AVG,
  AVG(i.Intake_BW) AS Intake_AVG_BW,
  COUNT(r.Food_Code) AS Recipes
FROM 
    Intake i
JOIN  Recipes r ON r.FCID_Code = i.FCID_Code
GROUP BY i.FCID_Code

## 2) Como você analisaria a correlação entre as métricas?

* Por exemplo, produtos mais populares são mais consumidos (em número de pessoas ou em quantidade)?
* Proponha uma ou mais queries para fazer esta análise

In [ ]:
--Para analisar a correlação entre as métricas, podemos usar a função CORR() do SQL, que retorna o coeficiente de correlação de Pearson entre duas colunas. O coeficiente de correlação varia de -1 a 1. Um valor próximo de 1 implica uma forte correlação positiva, enquanto um valor próximo de -1 implica uma forte correlação negativa. Um valor próximo de 0 implica que não há correlação.
--Exemplos de algumas consultas que analisam a correlação entre as métricas:
--1. Correlação entre a popularidade de um produto e a quantidade total consumida:
SELECT CORR(Popularidade, Intake_Sum) AS Correlation FROM Food_Metrics;
--2. Correlação entre a popularidade de um produto e a média de consumo:
SELECT CORR(Popularidade, Intake_AVG) AS Correlation FROM Food_Metrics;
--3. Correlação entre a popularidade de um produto e o número de receitas em que é usado:
SELECT CORR(Popularidade, Recipes) AS Correlation FROM Food_Metrics;

## 3) Podemos criar grupos de consumidores conforme um perfil?
* por exemplo, consumidores podem ser agrupados por alimentos que comem predominantemente?
* como você associaria grupos a classes?

In [ ]:
-- Sim, é possível agrupar consumidores com base em seus padrões de consumo de alimentos.
-- Com uma análise simples usando aSQL, poderíamos agrupar os consumidores com base no alimento que mais consomem:
SELECT SeqN, FCID_Code, MAX(Intake) as Max_Intake
FROM Intake
GROUP BY SeqN;
-- A consulta retorna o alimento que cada consumidor consome mais (em termos de gramas).E então agrupar os consumidores com base neste alimento.
-- Poderíamos associar os grupos a classe com base em algum tipo de critério ou regra dentro de padrões de consumo estabelecidos.

## 4) Que métricas podem ser analisadas para a comparação de perfis?
* escreva uma query SQL que calcule pelo menos uma métrica comparativa

In [ ]:
-- Calcula a diversidade de alimentos consumidos por cada consumidor:

SELECT SeqN, COUNT(DISTINCT FCID_Code) as Food_Diversity
FROM Intake
GROUP BY SeqN;